In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from math import pi

# Initialize simulator
sim = AerSimulator()

def run_and_print(qc, description):
    """Helper function to transpile, simulate, and print results"""
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=1024).result()
    counts = result.get_counts()
    print(f"\n--- {description} ---")
    print(qc.draw())
    print("Measurement Counts:", counts)

# ===============================================================
# 1. ORIGINAL CIRCUIT (reference)
# ===============================================================
qc_original = QuantumCircuit(1, 1)
qc_original.x(0)
qc_original.h(0)
qc_original.s(0)
qc_original.t(0)
qc_original.rz(0.5, 0)
qc_original.measure_all()
run_and_print(qc_original, "Original Circuit")

# ===============================================================
# 2. CHANGE GATE ORDER: Swap X and H gates
# ===============================================================
qc_swapped = QuantumCircuit(1, 1)
qc_swapped.h(0)   # H before X now
qc_swapped.x(0)
qc_swapped.s(0)
qc_swapped.t(0)
qc_swapped.rz(0.5, 0)
qc_swapped.measure_all()
run_and_print(qc_swapped, "Swapped X and H Gates")

print("\nExplanation:")
print("Gate order matters because quantum gates are *not commutative* — "
      "applying H then X gives a different state than X then H. "
      "This leads to different measurement probabilities.")

# ===============================================================
# 3. VARY ROTATION ANGLE (Rz)
# ===============================================================
for theta in [pi/4, pi/2, pi]:
    qc_varied = QuantumCircuit(1, 1)
    qc_varied.x(0)
    qc_varied.h(0)
    qc_varied.s(0)
    qc_varied.t(0)
    qc_varied.rz(theta, 0)
    qc_varied.measure_all()
    run_and_print(qc_varied, f"Rz rotation θ = {theta:.2f} radians")

print("\nObservation:")
print("As θ increases, the relative phase between |0> and |1> changes, "
      "which affects interference patterns and thus alters the final measurement probabilities.")

# ===============================================================
# 4. REMOVE HADAMARD GATE
# ===============================================================
qc_no_h = QuantumCircuit(1, 1)
qc_no_h.x(0)
# qc_no_h.h(0)  # Removed
qc_no_h.s(0)
qc_no_h.t(0)
qc_no_h.rz(0.5, 0)
qc_no_h.measure_all()
run_and_print(qc_no_h, "Circuit Without Hadamard Gate")

print("\nExplanation:")
print("Without the Hadamard gate, the qubit is not placed in superposition. "
      "Thus, the final measurement is more deterministic, unlike with H where probabilities are spread between |0> and |1>.")



--- Original Circuit ---
        ┌───┐┌───┐┌───┐┌───┐┌─────────┐ ░ ┌─┐
     q: ┤ X ├┤ H ├┤ S ├┤ T ├┤ Rz(0.5) ├─░─┤M├
        └───┘└───┘└───┘└───┘└─────────┘ ░ └╥┘
   c: 1/═══════════════════════════════════╬═
                                           ║ 
meas: 1/═══════════════════════════════════╩═
                                           0 
Measurement Counts: {'0 0': 504, '1 0': 520}

--- Swapped X and H Gates ---
        ┌───┐┌───┐┌───┐┌───┐┌─────────┐ ░ ┌─┐
     q: ┤ H ├┤ X ├┤ S ├┤ T ├┤ Rz(0.5) ├─░─┤M├
        └───┘└───┘└───┘└───┘└─────────┘ ░ └╥┘
   c: 1/═══════════════════════════════════╬═
                                           ║ 
meas: 1/═══════════════════════════════════╩═
                                           0 
Measurement Counts: {'1 0': 507, '0 0': 517}

Explanation:
Gate order matters because quantum gates are *not commutative* — applying H then X gives a different state than X then H. This leads to different measurement probabilities.

--- Rz rotation θ = 0